# Current-Corrected Vessel Velocities from EMSA Program AIS Messages

Exploration of processing of SalishSeaCast V2021-11 u & v current fields to calculate 
current-corrected vessel velocities from EMSA program AIS messages.

The `conda` environment in which this notebook runs is described in the `environment.yaml` file 
in this directory

In [1]:
from pathlib import Path

import numpy
import pandas
import xarray

Use the OPeNDAP capability of `xarray.open_dataset()` to load the u & v fields dataset metadata
from the SalishSeaCast ERDDAP server.

In [2]:
u_url = "https://salishsea.eos.ubc.ca/erddap/griddap/ubcSSg3DuGridFields1hV21-11"
v_url = "https://salishsea.eos.ubc.ca/erddap/griddap/ubcSSg3DvGridFields1hV21-11"

u_ds = xarray.open_dataset(u_url)
v_ds = xarray.open_dataset(v_url)

In [3]:
u_ds

<xarray.Dataset>
Dimensions:    (time: 150120, depth: 40, gridY: 898, gridX: 398)
Coordinates:
  * time       (time) datetime64[ns] 2007-01-01T00:30:00 ... 2024-02-15T23:30:00
  * depth      (depth) float32 0.5 1.5 2.5 3.5 4.5 ... 360.7 387.6 414.5 441.5
  * gridY      (gridY) int16 0 1 2 3 4 5 6 7 ... 890 891 892 893 894 895 896 897
  * gridX      (gridX) int16 0 1 2 3 4 5 6 7 ... 390 391 392 393 394 395 396 397
Data variables:
    uVelocity  (time, depth, gridY, gridX) float32 ...
Attributes: (12/26)
    acknowledgement:           MEOPAR, Ocean Networks Canada (ONC), Digital R...
    cdm_data_type:             Grid
    comment:                   If you use this dataset in your research,\nple...
    Conventions:               CF-1.6, COARDS, ACDD-1.3
    creator_email:             sallen@eoas.ubc.ca
    creator_name:              SalishSeaCast Project Contributors
    ...                        ...
    testOutOfDate:             now-16hours
    time_coverage_end:         2024-02-15T23:30:00Z
    time_coverage_start:       2007-01-01T00:30:00Z
    timeStamp:                 2024-Feb-15 17:27:07 GMT
    title:                     Green, Salish Sea, 3d u Grid Variable Fields, ...
    uuid:                      55fb25e8-5073-4934-9faa-78e0521d4dd5

In [4]:
v_ds

<xarray.Dataset>
Dimensions:    (time: 150120, depth: 40, gridY: 898, gridX: 398)
Coordinates:
  * time       (time) datetime64[ns] 2007-01-01T00:30:00 ... 2024-02-15T23:30:00
  * depth      (depth) float32 0.5 1.5 2.5 3.5 4.5 ... 360.7 387.6 414.5 441.5
  * gridY      (gridY) int16 0 1 2 3 4 5 6 7 ... 890 891 892 893 894 895 896 897
  * gridX      (gridX) int16 0 1 2 3 4 5 6 7 ... 390 391 392 393 394 395 396 397
Data variables:
    vVelocity  (time, depth, gridY, gridX) float32 ...
Attributes: (12/26)
    acknowledgement:           MEOPAR, Ocean Networks Canada (ONC), Digital R...
    cdm_data_type:             Grid
    comment:                   If you use this dataset in your research,\nple...
    Conventions:               CF-1.6, COARDS, ACDD-1.3
    creator_email:             sallen@eoas.ubc.ca
    creator_name:              SalishSeaCast Project Contributors
    ...                        ...
    testOutOfDate:             now-16hours
    time_coverage_end:         2024-02-15T23:30:00Z
    time_coverage_start:       2007-01-01T00:30:00Z
    timeStamp:                 2024-Feb-15 17:27:10 GMT
    title:                     Green, Salish Sea, 3d v Grid Variable Fields, ...
    uuid:                      363b7598-3fff-456c-98e7-c69ec9059980

Load the relevant columns from the AIS Excel file
using code from Peter's `Salish_Sea_SpeedThruWater.ipynb` notebook.

In [5]:
ais_data_path = Path("AIS_data.xlsx")

ais_df = pandas.read_excel(
    ais_data_path,
    usecols=['created','identity_id','speed','course','lat','lon','name','heading'],
    dtype={'lat': float,'lon': float}
)

In [6]:
ais_df

,created,identity_id,speed,course,lat,lon,name,heading
0,2024-01-16 09:44:52,636021537,9.9,188,48.530888,-123.218700,LAKE ANNECY,172.0
1,2024-01-16 07:56:44,357359000,6.4,16,48.643042,-123.231585,FEDERAL ILLINOIS,342.0
2,2024-01-16 07:53:52,563128300,5.8,14,48.470163,-123.168853,KAYING,347.0
3,2024-01-16 07:53:55,636019760,8.7,20,48.488953,-123.165610,GSL CHRISTEN,341.0
4,2024-01-16 07:59:55,255806029,9.3,139,48.785058,-123.017447,MSC NITYA B,224.0
...,...,...,...,...,...,...,...,...
1787,2024-01-16 07:47:51,563128300,5.7,354,48.451770,-123.166107,KAYING,5.0
1788,2024-01-15 16:14:52,256402000,7.4,14,48.623423,-123.232083,SATURALO,345.0
1789,2024-01-15 16:14:45,538007088,6.8,14,48.476035,-123.164172,BILLY JIM,341.0
1790,2024-01-16 08:23:45,357359000,6.8,293,48.712672,-123.190693,FEDERAL ILLINOIS,68.0


The SalishSeaCast grid region of interest for the QENTOL, YEN W̱SÁNEĆ Marine Guardians is
x = [220, 297] and y = [283, 350].

In [7]:
x_slice = slice(220, 297+1)
y_slice = slice(283, 350+1)

An alternative would be to grab the min/max lon/lat values from the AIS messages Excel file and use
https://github.com/SalishSeaCast/grid/blob/main/grid_from_lat_lon_mask999.nc to look up the corresponding
model grid region corners.

Use the creation times of the AIS messages to calculate the time slice we want the u & v fields for.
They are UTC, as are the SalishSeaCast model times.

In [8]:
time_min = ais_df["created"].min()
time_max = ais_df["created"].max()

time_min, time_max


(Timestamp('2024-01-15 16:11:46'), Timestamp('2024-01-17 10:29:54'))

In [9]:
u_surface = (
    u_ds
    .sel(time=slice(time_min, time_max), gridY=y_slice, gridX=x_slice)
    .sel(depth=0, method="nearest")
    )

u_surface

<xarray.Dataset>
Dimensions:    (time: 42, gridY: 69, gridX: 79)
Coordinates:
  * time       (time) datetime64[ns] 2024-01-15T16:30:00 ... 2024-01-17T09:30:00
    depth      float32 0.5
  * gridY      (gridY) int16 283 284 285 286 287 288 ... 346 347 348 349 350 351
  * gridX      (gridX) int16 220 221 222 223 224 225 ... 293 294 295 296 297 298
Data variables:
    uVelocity  (time, gridY, gridX) float32 ...
Attributes: (12/26)
    acknowledgement:           MEOPAR, Ocean Networks Canada (ONC), Digital R...
    cdm_data_type:             Grid
    comment:                   If you use this dataset in your research,\nple...
    Conventions:               CF-1.6, COARDS, ACDD-1.3
    creator_email:             sallen@eoas.ubc.ca
    creator_name:              SalishSeaCast Project Contributors
    ...                        ...
    testOutOfDate:             now-16hours
    time_coverage_end:         2024-02-15T23:30:00Z
    time_coverage_start:       2007-01-01T00:30:00Z
    timeStamp:                 2024-Feb-15 17:27:07 GMT
    title:                     Green, Salish Sea, 3d u Grid Variable Fields, ...
    uuid:                      55fb25e8-5073-4934-9faa-78e0521d4dd5

In [10]:
v_surface = (
    v_ds
    .sel(time=slice(time_min, time_max), gridY=y_slice, gridX=x_slice)
    .sel(depth=0, method="nearest")
    )

v_surface

<xarray.Dataset>
Dimensions:    (time: 42, gridY: 69, gridX: 79)
Coordinates:
  * time       (time) datetime64[ns] 2024-01-15T16:30:00 ... 2024-01-17T09:30:00
    depth      float32 0.5
  * gridY      (gridY) int16 283 284 285 286 287 288 ... 346 347 348 349 350 351
  * gridX      (gridX) int16 220 221 222 223 224 225 ... 293 294 295 296 297 298
Data variables:
    vVelocity  (time, gridY, gridX) float32 ...
Attributes: (12/26)
    acknowledgement:           MEOPAR, Ocean Networks Canada (ONC), Digital R...
    cdm_data_type:             Grid
    comment:                   If you use this dataset in your research,\nple...
    Conventions:               CF-1.6, COARDS, ACDD-1.3
    creator_email:             sallen@eoas.ubc.ca
    creator_name:              SalishSeaCast Project Contributors
    ...                        ...
    testOutOfDate:             now-16hours
    time_coverage_end:         2024-02-15T23:30:00Z
    time_coverage_start:       2007-01-01T00:30:00Z
    timeStamp:                 2024-Feb-15 17:27:10 GMT
    title:                     Green, Salish Sea, 3d v Grid Variable Fields, ...
    uuid:                      363b7598-3fff-456c-98e7-c69ec9059980

Checking the model times selected:

In [11]:
u_surface.time[0], u_surface.time[-1]

(<xarray.DataArray 'time' ()>
 array('2024-01-15T16:30:00.000000000', dtype='datetime64[ns]')
 Coordinates:
     time     datetime64[ns] 2024-01-15T16:30:00
     depth    float32 0.5
 Attributes:
     _CoordinateAxisType:    Time
     actual_range:           [1.1676114e+09 1.7080398e+09]
     axis:                   T
     comment:                time values are UTC at the centre of the interval...
     coverage_content_type:  modelResult
     ioos_category:          Time
     long_name:              Time axis
     standard_name:          time
     time_origin:            01-JAN-1970 00:00:00,
 <xarray.DataArray 'time' ()>
 array('2024-01-17T09:30:00.000000000', dtype='datetime64[ns]')
 Coordinates:
     time     datetime64[ns] 2024-01-17T09:30:00
     depth    float32 0.5
 Attributes:
     _CoordinateAxisType:    Time
     actual_range:           [1.1676114e+09 1.7080398e+09]
     axis:                   T
     comment:                time values are UTC at the centre of the interval.

There is probably room for refinement of the time slice limits choice.
In this case, the hour-averaged current components at 16:30 are a reasonable choice for the
`time_min` value of 16:11:46.
On the other hand, the 10:30 field values might be a better choice than 09:30 for the `time_max` value
of 10:29:54.
That refinement would have to be done by rounding the AIS time values because we can't use
`.sel(..., method="nearest")` when the selection is a slice.

Another option would be to interpolate the current values to the AIS time stamps from the 
hour-averaged model values at the 30 minute mark of the hours before and after the AIS
time stamps.

Two useful functions from [https://github.com/SalishSeaCast/tools/blob/main/SalishSeaTools/salishsea_tools/viz_tools.py](https://github.com/SalishSeaCast/tools/blob/main/SalishSeaTools/salishsea_tools/viz_tools.py).
Copy/pasted here to avoid installing a bunch of dependencies that the 
SalishSeaTools package needs that are irrelevant to this analysis.

In [12]:
def unstagger_xarray(qty, index):
    """Interpolate u, v, or w component values to values at grid cell centres.

    Named indexing requires that input arrays are XArray DataArrays.

    :arg qty: u, v, or w component values
    :type qty: :py:class:`xarray.DataArray`

    :arg index: index name along which to centre
        (generally one of 'gridX', 'gridY', or 'depth')
    :type index: str

    :returns qty: u, v, or w component values at grid cell centres
    :rtype: :py:class:`xarray.DataArray`
    """

    qty = (qty + qty.shift(**{index: 1})) / 2

    return qty

In [13]:
def rotate_vel(u_in, v_in, origin='grid'):
    """Rotate u and v component values to either E-N or model grid.

    The origin argument sets the input coordinates ('grid' or 'map')

    :arg u_in: u velocity component values
    :type u_in: :py:class:`numpy.ndarray`

    :arg v_in: v velocity component values
    :type v_in: :py:class:`numpy.ndarray`

    :arg origin: Input coordinate system
                 (either 'grid' or 'map', output will be the other)
    :type origin: str

    :returns u_out, v_out: rotated u and v component values
    :rtype: :py:class:`numpy.ndarray`
    """

    # Determine rotation direction
    if   origin == 'grid':
        fac =  1
    elif origin == 'map':
        fac = -1
    else:
        raise ValueError('Invalid origin value: {origin}'.format(
            origin=origin))

    # Rotate velocities
    theta_rad = 29 * numpy.pi / 180

    u_out = u_in * numpy.cos(theta_rad) - fac * v_in * numpy.sin(theta_rad)
    v_out = u_in * numpy.sin(theta_rad) * fac + v_in * numpy.cos(theta_rad)

    return u_out, v_out

Interpolate u and v surface component fields to the T grid point locations
at the centres of the model grid cells.
This is necessary because the model calculates the u components on the east faces
of the grid cells,
and the v components on the north faces.

This step triggers loading of the fields from the ERDDAP server,
in contrast to earlier steps that loaded only metadata.
The time for this operation is probably dominated by network bandwidth and,
to a lesser extent,
load on the ERDDAP server.
For my laptop on a nominal 1 Gbps service in Vancouver
(off-campus, so "farther" from the server that is in the university data centre),
this take around 1 minute.

In [14]:
u_surface_t = unstagger_xarray(u_surface, "gridX")
v_surface_t = unstagger_xarray(v_surface, "gridY")

In [15]:
u_surface_t

<xarray.Dataset>
Dimensions:    (time: 42, gridY: 69, gridX: 79)
Coordinates:
  * time       (time) datetime64[ns] 2024-01-15T16:30:00 ... 2024-01-17T09:30:00
    depth      float32 0.5
  * gridY      (gridY) int16 283 284 285 286 287 288 ... 346 347 348 349 350 351
  * gridX      (gridX) int16 220 221 222 223 224 225 ... 293 294 295 296 297 298
Data variables:
    uVelocity  (time, gridY, gridX) float32 nan 0.3744 0.3729 ... 0.2927 0.3217

In [16]:
v_surface_t

<xarray.Dataset>
Dimensions:    (time: 42, gridY: 69, gridX: 79)
Coordinates:
  * time       (time) datetime64[ns] 2024-01-15T16:30:00 ... 2024-01-17T09:30:00
    depth      float32 0.5
  * gridY      (gridY) int16 283 284 285 286 287 288 ... 346 347 348 349 350 351
  * gridX      (gridX) int16 220 221 222 223 224 225 ... 293 294 295 296 297 298
Data variables:
    vVelocity  (time, gridY, gridX) float32 nan nan nan ... -0.5817 -0.7362

Rotate the u and v fields from the model grid orientation to east-north orientation.

In [22]:
u_surface_east, v_surface_north = rotate_vel(u_surface_t.uVelocity, v_surface_t.vVelocity, origin="grid")

In [23]:
u_surface_east

<xarray.DataArray (time: 42, gridY: 69, gridX: 79)>
array([[[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,  0.2315875 ,  0.3469875 , ...,  0.        ,
          0.        ,  0.        ],
        [        nan,  0.28496766,  0.38496453, ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [        nan,  0.        ,  0.        , ...,  0.74129343,
          0.6197679 ,  0.4939649 ],
        [        nan,  0.        ,  0.        , ...,  0.720486  ,
          0.7189088 ,  0.79182917],
        [        nan,  0.        ,  0.00719424, ...,  0.9700524 ,
          0.94274426,  0.82961065]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan, -0.34400696, -0.23228225, ...,  0.        ,
          0.        ,  0.        ],
        [        nan, -0.2837641 , -0.1794202 , ...,  0.        ,
          0.        ,  0.        ],
...
        [        nan,  0.        ,  0.        , ...,  0.6736995 ,
          0.36013052,  0.45279592],
        [        nan,  0.        ,  0.        , ...,  0.7081553 ,
          0.7459729 ,  0.36564928],
        [        nan,  0.        ,  0.01650329, ...,  0.38451403,
          0.43875244,  0.32453772]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan, -0.34210533, -0.28612885, ...,  0.        ,
          0.        ,  0.        ],
        [        nan, -0.35707211, -0.31621227, ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [        nan,  0.        ,  0.        , ...,  0.5952625 ,
          0.4645608 ,  0.5983944 ],
        [        nan,  0.        ,  0.        , ...,  0.6615583 ,
          0.65877974,  0.5823688 ],
        [        nan,  0.        ,  0.01067099, ...,  0.4195127 ,
          0.5380049 ,  0.63828695]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 2024-01-15T16:30:00 ... 2024-01-17T09:30:00
  * gridY    (gridY) int16 283 284 285 286 287 288 ... 346 347 348 349 350 351
  * gridX    (gridX) int16 220 221 222 223 224 225 ... 293 294 295 296 297 298
    depth    float32 0.5

In [25]:
v_surface_north

<xarray.DataArray (time: 42, gridY: 69, gridX: 79)>
array([[[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan, -0.02799135, -0.0142481 , ...,  0.        ,
          0.        ,  0.        ],
        [        nan, -0.0852195 , -0.18514201, ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [        nan,  0.        ,  0.        , ..., -0.3253141 ,
          0.19520643, -0.08749449],
        [        nan,  0.        ,  0.        , ..., -0.40991217,
          0.10555251,  0.31533718],
        [        nan,  0.        , -0.0078032 , ...,  0.4109808 ,
          0.17815787,  0.34804317]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan, -1.0872142 , -1.1442014 , ...,  0.        ,
          0.        ,  0.        ],
        [        nan, -1.1990765 , -1.2289194 , ...,  0.        ,
          0.        ,  0.        ],
...
        [        nan,  0.        ,  0.        , ..., -1.2782657 ,
         -1.1010246 , -0.7348444 ],
        [        nan,  0.        ,  0.        , ..., -0.6960099 ,
         -0.62727714, -0.25716215],
        [        nan,  0.        , -0.00479732, ..., -0.34574753,
         -0.6019131 , -0.34530568]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan, -1.1117225 , -1.0695238 , ...,  0.        ,
          0.        ,  0.        ],
        [        nan, -1.1404314 , -1.1476582 , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [        nan,  0.        ,  0.        , ..., -1.0658243 ,
         -0.8387407 , -0.7673236 ],
        [        nan,  0.        ,  0.        , ..., -0.54391253,
         -0.46423978, -0.4771874 ],
        [        nan,  0.        , -0.0045265 , ..., -0.2815583 ,
         -0.36688742, -0.48787194]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 2024-01-15T16:30:00 ... 2024-01-17T09:30:00
  * gridY    (gridY) int16 283 284 285 286 287 288 ... 346 347 348 349 350 351
  * gridX    (gridX) int16 220 221 222 223 224 225 ... 293 294 295 296 297 298
    depth    float32 0.5

The `nan` values in the top row and left column of these arrays are an artefact from the
interpolation to the model T grid point locations.

Use code from Peter's `Salish_Sea_SpeedThruWater.ipynb` notebook to process the AIS messages
to prepare them for SalishSeaCast current corrections.

Add a column to express AIS course field
(measured in degrees CCW from North)
as a traditional compass bearing
(to match same co-ordinates as AIS heading field):

In [40]:
ais_df = ais_df.assign(true_course=360-ais_df.course)

ais_df

,created,identity_id,speed,course,lat,lon,name,heading,true_course
0,2024-01-16 09:44:52,636021537,9.9,188,48.530888,-123.218700,LAKE ANNECY,172.0,172
1,2024-01-16 07:56:44,357359000,6.4,16,48.643042,-123.231585,FEDERAL ILLINOIS,342.0,344
2,2024-01-16 07:53:52,563128300,5.8,14,48.470163,-123.168853,KAYING,347.0,346
3,2024-01-16 07:53:55,636019760,8.7,20,48.488953,-123.165610,GSL CHRISTEN,341.0,340
4,2024-01-16 07:59:55,255806029,9.3,139,48.785058,-123.017447,MSC NITYA B,224.0,221
...,...,...,...,...,...,...,...,...,...
1787,2024-01-16 07:47:51,563128300,5.7,354,48.451770,-123.166107,KAYING,5.0,6
1788,2024-01-15 16:14:52,256402000,7.4,14,48.623423,-123.232083,SATURALO,345.0,346
1789,2024-01-15 16:14:45,538007088,6.8,14,48.476035,-123.164172,BILLY JIM,341.0,346
1790,2024-01-16 08:23:45,357359000,6.8,293,48.712672,-123.190693,FEDERAL ILLINOIS,68.0,67


Add a column to express AIS course in cartesian coordinates
(measured in degrees CCW from East):

In [41]:
ais_df = ais_df.assign(cartesian_course=((ais_df.course+90)%360))

ais_df

,created,identity_id,speed,course,lat,lon,name,heading,true_course,cartesian_course
0,2024-01-16 09:44:52,636021537,9.9,188,48.530888,-123.218700,LAKE ANNECY,172.0,172,278
1,2024-01-16 07:56:44,357359000,6.4,16,48.643042,-123.231585,FEDERAL ILLINOIS,342.0,344,106
2,2024-01-16 07:53:52,563128300,5.8,14,48.470163,-123.168853,KAYING,347.0,346,104
3,2024-01-16 07:53:55,636019760,8.7,20,48.488953,-123.165610,GSL CHRISTEN,341.0,340,110
4,2024-01-16 07:59:55,255806029,9.3,139,48.785058,-123.017447,MSC NITYA B,224.0,221,229
...,...,...,...,...,...,...,...,...,...,...
1787,2024-01-16 07:47:51,563128300,5.7,354,48.451770,-123.166107,KAYING,5.0,6,84
1788,2024-01-15 16:14:52,256402000,7.4,14,48.623423,-123.232083,SATURALO,345.0,346,104
1789,2024-01-15 16:14:45,538007088,6.8,14,48.476035,-123.164172,BILLY JIM,341.0,346,104
1790,2024-01-16 08:23:45,357359000,6.8,293,48.712672,-123.190693,FEDERAL ILLINOIS,68.0,67,23


Separate AIS speed and course into speed along North and East axes:

In [44]:
def ais_velocity_components(row):
    ais_vel_n = numpy.sin(numpy.deg2rad(row.cartesian_course))*row.speed
    ais_vel_e = numpy.cos(numpy.deg2rad(row.cartesian_course))*row.speed
    return ais_vel_e, ais_vel_n

In [45]:
ais_df['aisVelE'], ais_df['aisVelN'] =zip(*ais_df.apply(ais_velocity_components,axis=1))

ais_df

,created,identity_id,speed,course,lat,lon,name,heading,true_course,cartesian_course,aisVelE,aisVelN
0,2024-01-16 09:44:52,636021537,9.9,188,48.530888,-123.218700,LAKE ANNECY,172.0,172,278,1.377814,-9.803654
1,2024-01-16 07:56:44,357359000,6.4,16,48.643042,-123.231585,FEDERAL ILLINOIS,342.0,344,106,-1.764079,6.152075
2,2024-01-16 07:53:52,563128300,5.8,14,48.470163,-123.168853,KAYING,347.0,346,104,-1.403147,5.627715
3,2024-01-16 07:53:55,636019760,8.7,20,48.488953,-123.165610,GSL CHRISTEN,341.0,340,110,-2.975575,8.175326
4,2024-01-16 07:59:55,255806029,9.3,139,48.785058,-123.017447,MSC NITYA B,224.0,221,229,-6.101349,-7.018799
...,...,...,...,...,...,...,...,...,...,...,...,...
1787,2024-01-16 07:47:51,563128300,5.7,354,48.451770,-123.166107,KAYING,5.0,6,84,0.595812,5.668775
1788,2024-01-15 16:14:52,256402000,7.4,14,48.623423,-123.232083,SATURALO,345.0,346,104,-1.790222,7.180188
1789,2024-01-15 16:14:45,538007088,6.8,14,48.476035,-123.164172,BILLY JIM,341.0,346,104,-1.645069,6.598011
1790,2024-01-16 08:23:45,357359000,6.8,293,48.712672,-123.190693,FEDERAL ILLINOIS,68.0,67,23,6.259433,2.656972


Load high resolution mapping of lons/lats to SalishSeaCast model grid y/x indices (j/i).
This mapping has longitude steps of 0.0005291 degrees,
and latitude steps of 0.0003125 degrees.
Using it to look up grid j/i values that correspond to lon/lat values is substantially faster than
using
[`salishsea_tools.geo_tools.find_closest_model_point()`](https://salishsea-meopar-tools.readthedocs.io/en/latest/SalishSeaTools/api.html#salishsea_tools.geo_tools.find_closest_model_point).

In [ ]:
ji_lonlat_map_path = Path("/SalishSeaCast/grid/grid_from_lat_lon_mask999.nc")

ji_lonlat_map = xarray.open_dataset(ji_lonlat_map_path)

ji_lonlat_map

<xarray.Dataset>
Dimensions:  (lons: 9605, lats: 13585)
Coordinates:
  * lons     (lons) float64 -126.4 -126.4 -126.4 -126.4 ... -121.3 -121.3 -121.3
  * lats     (lats) float64 46.86 46.86 46.86 46.86 ... 51.1 51.1 51.1 51.1
Data variables:
    jj       (lats, lons) int16 ...
    ii       (lats, lons) int16 ...
Attributes:
    acknowledgements:      Based on find_closest point
    creator_email:         sallen@eoas.ubc.ca
    creator_name:          Salish Sea MEOPAR Project Contributors
    creator_url:           https://salishsea-meopar-docs.readthedocs.org/
    institution:           UBC EOAS
    institution_fullname:  Earth, Ocean & Atmospheric Sciences, University of...
    title:                 Grid position from lat/lons
    history:               [2021-02-25] File creation.
    Mask:                  from geometry and then corrected based on results,...

Find nearest SalishSeaCast grid point for each AIS message lon/lat:

In [67]:
def lonlat_to_ji(row):
    j = ji_lonlat_map.jj.sel(lons=row.lon, lats=row.lat, method="nearest").item()
    i = ji_lonlat_map.ii.sel(lons=row.lon, lats=row.lat, method="nearest").item()
    return j, i

In [68]:
ais_df['grid_y'], ais_df['grid_x'] = zip(*ais_df.apply(lonlat_to_ji, axis=1))

ais_df

,created,identity_id,speed,course,lat,lon,name,heading,true_course,cartesian_course,aisVelE,aisVelN,grid_y,grid_x
0,2024-01-16 09:44:52,636021537,9.9,188,48.530888,-123.218700,LAKE ANNECY,172.0,172,278,1.377814,-9.803654,310,233
1,2024-01-16 07:56:44,357359000,6.4,16,48.643042,-123.231585,FEDERAL ILLINOIS,342.0,344,106,-1.764079,6.152075,333,245
2,2024-01-16 07:53:52,563128300,5.8,14,48.470163,-123.168853,KAYING,347.0,346,104,-1.403147,5.627715,295,233
3,2024-01-16 07:53:55,636019760,8.7,20,48.488953,-123.165610,GSL CHRISTEN,341.0,340,110,-2.975575,8.175326,299,236
4,2024-01-16 07:59:55,255806029,9.3,139,48.785058,-123.017447,MSC NITYA B,224.0,221,229,-6.101349,-7.018799,345,294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1787,2024-01-16 07:47:51,563128300,5.7,354,48.451770,-123.166107,KAYING,5.0,6,84,0.595812,5.668775,291,231
1788,2024-01-15 16:14:52,256402000,7.4,14,48.623423,-123.232083,SATURALO,345.0,346,104,-1.790222,7.180188,329,243
1789,2024-01-15 16:14:45,538007088,6.8,14,48.476035,-123.164172,BILLY JIM,341.0,346,104,-1.645069,6.598011,296,235
1790,2024-01-16 08:23:45,357359000,6.8,293,48.712672,-123.190693,FEDERAL ILLINOIS,68.0,67,23,6.259433,2.656972,344,260


In [61]:
u_surface_east

<xarray.DataArray (time: 42, gridY: 69, gridX: 79)>
array([[[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,  0.2315875 ,  0.3469875 , ...,  0.        ,
          0.        ,  0.        ],
        [        nan,  0.28496766,  0.38496453, ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [        nan,  0.        ,  0.        , ...,  0.74129343,
          0.6197679 ,  0.4939649 ],
        [        nan,  0.        ,  0.        , ...,  0.720486  ,
          0.7189088 ,  0.79182917],
        [        nan,  0.        ,  0.00719424, ...,  0.9700524 ,
          0.94274426,  0.82961065]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan, -0.34400696, -0.23228225, ...,  0.        ,
          0.        ,  0.        ],
        [        nan, -0.2837641 , -0.1794202 , ...,  0.        ,
          0.        ,  0.        ],
...
        [        nan,  0.        ,  0.        , ...,  0.6736995 ,
          0.36013052,  0.45279592],
        [        nan,  0.        ,  0.        , ...,  0.7081553 ,
          0.7459729 ,  0.36564928],
        [        nan,  0.        ,  0.01650329, ...,  0.38451403,
          0.43875244,  0.32453772]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan, -0.34210533, -0.28612885, ...,  0.        ,
          0.        ,  0.        ],
        [        nan, -0.35707211, -0.31621227, ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [        nan,  0.        ,  0.        , ...,  0.5952625 ,
          0.4645608 ,  0.5983944 ],
        [        nan,  0.        ,  0.        , ...,  0.6615583 ,
          0.65877974,  0.5823688 ],
        [        nan,  0.        ,  0.01067099, ...,  0.4195127 ,
          0.5380049 ,  0.63828695]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 2024-01-15T16:30:00 ... 2024-01-17T09:30:00
  * gridY    (gridY) int16 283 284 285 286 287 288 ... 346 347 348 349 350 351
  * gridX    (gridX) int16 220 221 222 223 224 225 ... 293 294 295 296 297 298
    depth    float32 0.5

In [66]:
(
    u_surface_east
    .sel(time=ais_df.created[0], method="nearest")
    .sel(gridY=ais_df.grid_y[0], gridX=ais_df.grid_x[0])
)


<xarray.DataArray ()>
array(0.12666363, dtype=float32)
Coordinates:
    time     datetime64[ns] 2024-01-16T09:30:00
    gridY    int16 310
    gridX    int16 233
    depth    float32 0.5

In [69]:
def model_point_current(row):
    point_u_surface_east = (
        u_surface_east
        .sel(time=row.created, method="nearest")
        .sel(gridY=row.grid_y, gridX=row.grid_x)
    )
    point_v_surface_north = (
        v_surface_north
        .sel(time=row.created, method="nearest")
        .sel(gridY=row.grid_y, gridX=row.grid_x)
    )
    return point_u_surface_east, point_v_surface_north
